In [1]:
!pip install ipynb

In [2]:
!pip install -U tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.3.0
    Uninstalling tensorflow-datasets-4.3.0:
      Successfully uninstalled tensorflow-datasets-4.3.0


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [4]:
import sys
sys.path.insert(0,"../usr/lib")

In [5]:
import datetime
import gc
from pathlib import Path

import keras_tuner as kt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from ipynb.fs.defs.data_pipeline import EXPORTED as data_pipeline
from tensorflow import keras

# Loading builders for datasets

In [6]:
RECIPES5K_DIR = "../input/foodnet/tfrecord/recipes5k/1.0.0"
FOOD101_DIR = "../input/foodnet/tfrecord/food101/1.1.0"
NUTRITION5K_DIR = "../input/foodnet/tfrecord/nutrition5k/1.0.0"


recipes5k_food101_builder = tfds.builder_from_directories([RECIPES5K_DIR, FOOD101_DIR])
nutrition5k_builder = tfds.builder_from_directory(NUTRITION5K_DIR)

In [7]:
recipes5k_food101_builder.info.splits["train"].num_examples

105826

In [8]:
nutrition5k_builder.info.splits["train"].num_examples

271407

In [9]:
def encode_category_ingredients(category_tensor, ingredients_tensor):
    category = str(category_tensor.numpy(), "utf-8")
    ingredients = str(ingredients_tensor.numpy(), "utf-8")
    one_hot_category_tensor = (
        data_pipeline.one_hot_encoder.get_category_one_hot_encoding(category)
    )
    one_hot_ingredients_tensor = (
        data_pipeline.one_hot_encoder.get_ingredients_one_hot_encoding(
            ingredients.split(",")
        )
    )
    return (
        tf.constant(one_hot_category_tensor, dtype=tf.uint8),
        tf.constant(one_hot_ingredients_tensor, dtype=tf.uint8),
    )

In [10]:
def parse_function(x):
    encoded = tf.py_function(
        encode_category_ingredients,
        [x["category"], x["ingredients"]],
        [tf.uint8, tf.uint8],
    )
    return x["image_raw"], {
        "category_output": encoded[0],
        "calorie_output": x["calorie"],
        "carbs_output": x["carbs"],
        "protein_output": x["protein"],
        "fat_output": x["fat"],
        "ingredients_output": encoded[1],
    }

## Creating dataset for training ingredients and nutrients only

In [11]:
# FULL DATASET
no_category_train, no_category_val = nutrition5k_builder.as_dataset(
    split=["train[:80%]", "train[80%:]"]
)

In [12]:
print(f"Total training size : {no_category_train.cardinality().numpy()}")
print(f"Total validation size : {no_category_val.cardinality().numpy()}")

Total training size : 217126
Total validation size : 54281


In [13]:
BATCH_SIZE = 256
no_category_train = (
    no_category_train.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_val = (
    no_category_val.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

## Creating dataset for training category, ingredients and nutrients

In [14]:
# FULL DATASET
(
    recipes5k_food101_train,
    recipes5k_food101_validation,
) = recipes5k_food101_builder.as_dataset(
    split=["train[:80%]", "train[80%:]"]
)

# Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
(
    nutrition5k_category_train,
    nutrition5k_category_val,
) = nutrition5k_builder.as_dataset(
    split=["train[:800]", "train[800:1000]"]
)

train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)

In [15]:
print(f"Total training size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")

Total training size : 85461
Total validation size : 21365


In [16]:
BATCH_SIZE = 256
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

# Model Development

In [17]:
class BaseModel:
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        self.input_shape = input_shape
        self.total_food_category = total_food_category
        self.total_ingredients_category = total_ingredients_category
        self.model = None
        self.name = name
        self.model_config_name = model_config_name

    def build(
        self,
        independent_category_units,
        independent_ingredients_units,
        independent_protein_units,
        independent_calorie_units,
        independent_carbs_units,
        independent_fat_units,
        shared_units,
    ):
        raise NotImplementedError

    def compile_model (self, optimizer):
        assert self.model is not None
        category_classification_loss = keras.losses.CategoricalCrossentropy()
        calorie_regression_loss = keras.losses.MeanAbsoluteError()
        carbs_regression_loss = keras.losses.MeanAbsoluteError()
        protein_regression_loss = keras.losses.MeanAbsoluteError()
        fat_regression_loss = keras.losses.MeanAbsoluteError()
        ingredient_multilabel_loss = keras.losses.BinaryCrossentropy()
        category_classification_metrics = (
            [
                keras.metrics.CategoricalAccuracy(name="acc"),
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall")
            ],
        )

        calorie_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        carbs_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        protein_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        fat_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        ingredient_multilabel_metrics = (
            [
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall")
            ],
        )
        category_classification_loss_weights = 1.0
        ingredient_multilabel_loss_weights = 1.0
        calorie_regression_loss_weights = 1.0
        carbs_regression_loss_weights = 1.0
        protein_regression_loss_weights = 1.0
        fat_regression_loss_weights = 1.0

        self.model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )

    def freeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def freeze_convolution_base(self):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetB1")
        submodel.trainable = False

    def get_callbacks(self):
        tensorboard_dir = f"./models/logs/{self.name}/{self.model_config_name}"
        checkpoint_dir = f"./temp/checkpoint/{self.name}/{self.model_config_name}"
        checkpoint_model_path = checkpoint_dir + "/model"
        checkpoint_weights_path = checkpoint_dir + "/weights/ckpt"
        checkpoint_model_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_model_path, monitor="val_loss", save_best_only=True, mode="min"
        )
        checkpoint_weights_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_weights_path, monitor="val_loss", save_best_only=True, save_weights_only=True, mode="min"
        )
        reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1
        )
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=8)
        return [
            tensorboard_callback,
            early_stopping_callback,
            checkpoint_model_callback,
            checkpoint_weights_callback,
            reduce_lr_callback,
        ]

    ## Layers from bottom to top (classifier)  ##
    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, input_tensor):
        raise NotImplementedError

    def get_convolution_block(self):
        raise NotImplementedError

    def get_shared_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            num_units[0], activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(
        self, input_tensor, total_categories, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(
            num_units[0], activation="relu", name="category_dense_1"
        )(input_layer)
        x = keras.layers.BatchNormalization()(x)
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output_layer"
        )(x)
        output_model = keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )
        return output_model

    def get_calorie_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="calorie_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="calorie_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="carbs_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="carbs_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="protein_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="protein_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        protein_regression_layers = keras.layers.Dense(1, name="protein_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="fat_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="fat_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        fat_regression_layers = keras.layers.Dense(1, name="fat_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(
        self, input_tensor, total_ingredients, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[1], activation="relu", name="ingredients_dense_2"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[2], activation="relu", name="ingredients_dense_3"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output_layer"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    ## End of layers  ##

    def load_model(self, path):
        self.model = keras.models.load_model(path)

    def print_summary(self):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        for layer in self.model.layers:
            if "input" in layer.name:
                continue
            try :
                layer.summary()
            except Exception:
                print(layer.name)
            print()
        self.model.summary()

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def train_model(self, **kwargs):
        assert (
            self.model is not None
        ), "No model found."
        return self.model.fit(**kwargs, callbacks=self.get_callbacks())

    def unfreeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = True

    def unfreeze_convolution_base(self, fine_tune_at=0):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetb1")
        submodel.trainable = True
        for layer in submodel.layers[:fine_tune_at]:
            layer.trainable = False

## Flat Model

In [18]:
class FlatModel(BaseModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            name,
            model_config_name,
        )

    def build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
    ):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        prev_layer = self.get_convolution_block()(prev_layer)
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model

### EfficientNetB1 (Best)

In [19]:
class FlatEfficientNetB1Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on ingredients dataset (nutrition5k)

In [20]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalIngredients",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_efficientnet.freeze_category_classification_layers()
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

27033600/27018416 [==============================] - 0s 0us/step
Model: "augmentation_layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 224, 224, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 224, 224, 3)       0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

Model: "preprocessing_layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_____________________________

In [ ]:
flat_efficientnet.train_model(
    x=no_category_train, validation_data=no_category_val, epochs=30, verbose=2, initial_epoch=12
)

#### Train on main dataset (recipes5k + food101 + nutrition5k) with weights transfer from ingredients dataset

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategory",
)

flat_efficientnet.load_model("../input/foodnet/finalCategory/model")
flat_efficientnet.unfreeze_category_classification_layers()
flat_efficientnet.compile_model(keras.optimizers.Adam(learning_rate = 1e-4))

flat_efficientnet.print_summary()

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=50, verbose=2, initial_epoch = 26
)

#### Fine tune the convolution base with weights transfer from ingredients dataset

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransferFineTuned",
)
flat_efficientnet.load_model("./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer/model")
flat_efficientnet.unfreeze_convolution_base(117)
flat_efficientnet.compile_model(keras.optimizers.Adam(learning_rate = 1e-5))
flat_efficientnet.print_summary()

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=100, verbose=2, initial_epoch=25
)

#### Train on main dataset (recipes5k + food101 + nutrition5k) without weights transfer

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransfer",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=50, verbose=2
)

#### Fine tune the convolution base without weights transfer

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransferFineTuned",
)
flat_efficientnet.load_model("./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer/model")
flat_efficientnet.unfreeze_convolution_base(117)
flat_efficientnet.compile_model(keras.optimizers.Adam(learning_rate = 1e-5))
flat_efficientnet.print_summary()

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=100, verbose=2, initial_epoch=25
)

### MobileNetv3

In [ ]:
class FlatMobileNetv3Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_mobilenetv3",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_mobilenetv3.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_mobilenetv3.compile_model(keras.optimizers.Adam())
flat_mobilenetv3.print_summary()

In [ ]:
flat_mobilenetv3.train_model(
    x=no_category_train, validation_data=no_category_val, epochs=2, verbose=1
)

### NASNet Mobile

In [ ]:
class FlatNASNetModel(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_nasnet",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.nasnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        nasnet_convolution_layers = keras.applications.nasnet.NASNetMobile(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        nasnet_convolution_layers.trainable = False
        return nasnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
flat_nasnet = FlatNASNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_nasnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_nasnet.compile_model(keras.optimizers.Adam())

flat_nasnet.print_summary()

In [ ]:
flat_nasnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=10, verbose=1
)

## Wide Slice Model

In [ ]:
class WideSliceModel(BaseModel):
    def build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
        overwrite_cached_model=True,
    ):
        if not overwrite_cached_model:
            assert (
                self.model is not None
            ), "There is no previous model found. Do you mean to build a new model ?"
            return self.model
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        conv_layer = self.get_convolution_block()(prev_layer)
        wide_slice_layer = self.get_wideslice_conv_layers(prev_layer)(prev_layer)
        prev_layer = keras.layers.Concatenate()([conv_layer, wide_slice_layer])
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model

    def get_wideslice_conv_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        rescale_layer = keras.layers.Rescaling(scale=1.0 / 255)(input_layer)
        wideslice_conv = keras.layers.Conv2D(filters=64, kernel_size=(5, 224))(
            rescale_layer
        )
        wideslice_conv = keras.layers.BatchNormalization()(wideslice_conv)
        wideslice_conv = keras.layers.ReLU()(wideslice_conv)
        wideslice_conv = keras.layers.GlobalMaxPool2D()(wideslice_conv)
        return keras.Model(
            inputs=input_layer, outputs=wideslice_conv, name="wideslice_conv"
        )

### EfficientNetB1

In [ ]:
class WideSliceEfficientNetModel(WideSliceModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "wideslice_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
wideslice_efficientnet = WideSliceEfficientNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransfer",
)

wideslice_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_protein_units=[64,32],
    independent_fat_units=[64,32],
    independent_carbs_units=[64,32],
    independent_calorie_units=[64, 32],
    independent_ingredients_units=[1024, 512, 256],
)
wideslice_efficientnet.compile_model(keras.optimizers.Adam())
wideslice_efficientnet.print_summary()

In [ ]:
wideslice_efficientnet.train_model(
    x=train_dataset, epochs=50, verbose=2, validation_data=validation_dataset
)

#### Transfer weights from fine tuned flat model

In [ ]:
fine_tuned_flat_efficientnet = keras.models.load_model("./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransferFineTuned/model")
fine_tuned_efficientnet_weights = fine_tuned_flat_efficientnet.get_layer("efficientnetb1").get_weights()

In [ ]:
wideslice_efficientnet = WideSliceEfficientNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransferFineTuned",
)

wideslice_efficientnet.load_model("./temp/checkpoint/wideslice_efficientnetB1/finalCategoryNoTransfer/model")
wideslice_efficientnet.unfreeze_convolution_base(117)
wideslice_efficientnet.model.get_layer("efficientnetb1").set_weights(fine_tuned_efficientnet_weights)
wideslice_efficientnet.compile_model(keras.optimizers.Adam(learning_rate = 1e-5))
wideslice_efficientnet.print_summary()

In [ ]:
wideslice_efficientnet.train_model(x=train_dataset, epochs=50, verbose=2, validation_data=validation_dataset, initial_epoch = 20)

# Testing

In [ ]:
import os
os.chdir(r'/kaggle/working')

!tar -czf temp.tar.gz ./temp

from IPython.display import FileLink

FileLink(r'temp.tar.gz')

In [ ]:
import shutil
shutil.rmtree("./models")
shutil.rmtree("./temp")

In [ ]:
from pathlib import Path
Path("./temp").mkdir()
Path("./models").mkdir()  
Path("./temp/checkpoint").mkdir()

In [ ]:
flat_efficientnet = keras.models.load_model("../input/foodnet/flat_efficientnetB1/finalCategoryNoTransferFineTuned/model")

In [ ]:
flat_efficientnet.evaluate(validation_dataset)

In [ ]:
flat_efficientnet.save("./finalFlatEfficientnetB1.h5")

In [ ]:
wideslice_efficientnet = keras.models.load_model("../input/foodnet/wideslice_efficientnetB1/finalCategoryNoTransferFineTuned/model")

In [ ]:
wideslice_efficientnet.evaluate(validation_dataset)

In [ ]:
wideslice_efficientnet.save("./finalWideSliceEfficientnetB1.h5")